In [1]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.Inshop import Inshop_Dataset
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from tqdm import *
import wandb

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
LOG_DIR = '.'
trn_dataset = dataset.load('market', '../../Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

In [4]:
dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = 50,
        shuffle = True,
        num_workers = 4,
        drop_last = True,
        pin_memory = True
    )

In [5]:
model = Resnet50(embedding_size=512, pretrained=True, is_norm=1, bn_freeze =1).cuda()

/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = losses.Proxy_Anchor(nb_classes = trn_dataset.nb_classes(), sz_embed = 512, mrg = 0.1, alpha = 32).cuda()

In [7]:
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters())))},
    {'params': model.model.embedding.parameters(), 'lr':float(1e-4) * 1},
]
param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

In [8]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [9]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [10]:
for epoch in range(0, 60):
    model.train()
    bn_freeze = True

    if bn_freeze:
            modules = model.model.modules()
            for m in modules: 
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()

    losses_per_epoch = []
    unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(dl_tr))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))

0it [00:00, ?it/s]/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Train Epoch: 0 [44/44 (98%)] Loss: 7.966811: : 44it [00:04,  9.89it/s] 
Train Epoch: 1 [44/44 (98%)] Loss: 7.550196: : 44it [00:12,  3.63it/s]
Train Epoch: 2 [44/44 (98%)] Loss: 7.102752: : 44it [00:11,  3.69it/s]
Train Epoch: 3 [44/44 (98%)] Loss: 6.702276: : 44it [00:11,  3.69it/s]
Train Epoch: 4 [44/44 (98%)] Loss: 5.151131: : 44it [00:11,  3.69it/s]
Train Epoch: 5 [44/44 (98%)] Loss: 5.675545: : 44it [00:11,  3.68it/s]
Train Epoch: 6 [44/44 (98%)] Loss: 5.242073: : 44it [00:11,  3.68it/s]
Train Epoch: 7 [44/44 (98%)] Loss: 5.262970: : 44it [00:11,  3.69it/s]
Train Epoch: 8 [44/44 (98%)] Loss: 4.065011: : 44it [00:11,  3.69it/s]
Train Epoch: 9 [44/